<B>Load the llama3 token from .env</B>

In [2]:
# huggingface-cli login   >>> we can use this onetime in the terminal 

# or throw this 
# import os
# from dotenv import load_dotenv

# load_dotenv()
# api_token = os.getenv("LLAMA_API_TOKEN")

In [1]:
# Check the GPU availability    
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
NVIDIA GeForce RTX 3060 Laptop GPU


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

def initialize_llama3(model_id="meta-llama/Meta-Llama-3-8B-Instruct", use_4bit=True):
    """
    Initialize LLaMA 3 model from Hugging Face using Transformers.
    
    Args:
        model_id (str): Hugging Face model ID (default: LLaMA 3 8B Instruct)
        use_4bit (bool): Whether to load the model using 4-bit quantization for memory efficiency

    Returns:
        tuple: (model, tokenizer, text-generation pipeline)
    """
    print(f"Loading model: {model_id}")

    # Load the tokenizer (no need to manually pass token)
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # Load the model
    if use_4bit:
        # Optional: requires `bitsandbytes` installed
        quantization_config = {
            "load_in_4bit": True,
            "bnb_4bit_compute_dtype": torch.bfloat16,
            "bnb_4bit_quant_type": "nf4",
            "bnb_4bit_use_double_quant": True,
        }
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            torch_dtype=torch.bfloat16,
            quantization_config=quantization_config,
        )
    else:
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            torch_dtype=torch.bfloat16
        )

    # Create a text-generation pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False
    )

    return model, tokenizer, pipe

# Example usage
model, tokenizer, llama3_pipe = initialize_llama3()

Loading model: meta-llama/Meta-Llama-3-8B-Instruct


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

c:\Users\aliay\OneDrive\Desktop\iTi\30 - GenAI\Day 2\Prompt-Engineering-With-llama3\env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aliay\.cache\huggingface\hub\models--meta-llama--Meta-Llama-3-8B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

In [3]:
import torch
print(torch.__version__)

2.1.2+cu118
